######################################################
# AB Testing (Bağımsız İki Örneklem T Testi)
######################################################


1. Hipotezleri Kur
2. Varsayım Kontrolü
   - 1. Normallik Varsayımı
   - 2. Varyans Homojenliği
 3. Hipotezin Uygulanması
   - p-value < 0.05 ise HO red.
   - 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
   - 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)
 Not:
 - Normallik sağlanmıyorsa direk 2 numara. Varyans homojenliği sağlanmıyorsa 1 numaraya arguman girilir.
 - Normallik incelemesi öncesi aykırı değer incelemesi ve düzeltmesi yapmak faydalı olabilir.


In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, \
    f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

############################
# 1. Hipotezi Kur
############################

In [2]:
# H0: M1 = M2
# H1: M1!= M2

df = sns.load_dataset("tips")
df.groupby("smoker").agg({"total_bill": "mean"})

# Pek bir fark yok gibi duruyor!

,total_bill
smoker,
Yes,20.75634
No,19.18828


############################
# 2. Varsayım Kontrolü
############################

 Normallik Varsayımı
 Varyans Homojenliği

In [3]:
#################################
# Normallik Varsayımı (SHAPIRO)
#################################

# Yani standart normal dağılıma uyup uymadığının kontrolü yapılır

test_stat, pvalue = shapiro(df.loc[df["smoker"] == "Yes", "total_bill"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# 0.0002 < 0.05 çıkmıştır.

# İpucu:
# p-value < ise 0.05'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.

Test Stat = 0.9367, p-value = 0.0002


In [4]:
test_stat, pvalue = shapiro(df.loc[df["smoker"] == "No", "total_bill"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# 0.0000 < 0.05 çıkmıştır.

# İpucu:
# p-value < ise 0.05'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.

Test Stat = 0.9045, p-value = 0.0000



H0 red edilmiştir ve Normal dağılıma uymadığı anlaşılmıştır.

##############################################
# Varyans Homojenligi Varsayımı - Levene Testi
###############################################

In [5]:
# 2 örneklemin dağılımın benzerliğine bakar. Benzer ise homojen denir.

#Hipotezler:
# H0: Varyanslar Homojendir
# H1: Varyanslar Homojen Değildir


test_stat, pvalue = levene(df.loc[df["smoker"] == "Yes", "total_bill"],
                           df.loc[df["smoker"] == "No", "total_bill"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# İpucu:
# p-value < ise 0.05 'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.

# 0.0452 < 0.05 küçük gelmiştir ve H0 rededilir.
# Yani Varyanslar, Homojen değildir.

Test Stat = 4.0537, p-value = 0.0452


############################
# 3. Hipotezin Uygulanması
############################


 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)

 Eğer normallik sağlanmazsa her türlü nonparametrik test yapacağız.
 Eger normallik sağlanır varyans homojenliği sağlanmazsa ne olacak?
 T test fonksiyonuna arguman gireceğiz.

In [6]:
# Burada Hipotez uygulamasında 2. seçenek uygulanır çünkü normallik ve varyans homojenliği sağlanmamıştır.
# Mannwhitneyu Testi:

test_stat, pvalue = mannwhitneyu(df.loc[df["smoker"] == "Yes", "total_bill"],
                                 df.loc[df["smoker"] == "No", "total_bill"])


print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 6511.5000, p-value = 0.1707


# Sonuç:
 0.1707 > 0.05 ise H0 rededilemez ve M1 = M2 hipotezi sağlanmış olur.
 Ortalamalar arasında bir fark yoktur istatiksel anlamda.

############################
# 3. Eğer varsayımlar sağlasaydı !
############################

In [7]:
# T testi yapılır:

test_stat, pvalue = ttest_ind(df.loc[df["smoker"] == "Yes", "total_bill"],
                              df.loc[df["smoker"] == "No", "total_bill"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# Sonuç:
# 0.1820 > 0.05 ise H0 rededilemez ve M1 = M2 hipotezi sağlanmış olur.
# Ortalamalar arasında bir fark yoktur istatiksel anlamda.

# İpucu:
# p-value < ise 0.05 'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.

Test Stat = 1.3384, p-value = 0.1820
